<a href="https://colab.research.google.com/github/ml2-picme/PicMe/blob/master/Model%20Comparison_more%20models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Methodendefinitionen

In [0]:
# Install the PyDrive wrapper & import libraries
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
from keras.applications.vgg16 import VGG16, preprocess_input

from PIL import Image
import numpy as np

from keras.applications.vgg16 import decode_predictions
import matplotlib.pyplot as plt

#import plotly.plotly as plotly

from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.graph_objs as go

# Define the Methods

localFilePath = "/root/data"

def reloadPictures():
  # Authenticate and create the PyDrive client
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # Delete the local directory
  !rm -r $localFilePath
  print("Deleted local directory")

  # Set a local directory and upload files from Google Drive
  local_root_path = os.path.expanduser(localFilePath)
  try:
    os.makedirs(local_root_path)
  except: pass

  def ListFolder(google_drive_id, destination):
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % google_drive_id}).GetList()
    counter = 0
    for f in file_list:
      # If it is a directory then, create the dicrectory and upload the file inside it
      if f['mimeType']=='application/vnd.google-apps.folder': 
        folder_path = os.path.join(destination, f['title'])
        os.makedirs(folder_path)
        print('* creating directory {}'.format(folder_path))
        ListFolder(f['id'], folder_path)
      else:
        fname = os.path.join(destination, f['title'])
        f_ = drive.CreateFile({'id': f['id']})
        f_.GetContentFile(fname)
        counter += 1
    print('* {} files were uploaded in {}'.format(counter, destination))

  # Opening and uploading the Drive Folder => this is the shared directory and should work for everyone who belongs to the project
  print("\nCalling the ListFolder method, that uploads the data...")
  ListFolder("1M7TDDTFoNOjsrlEMonWN858uQONUp9Pz", local_root_path)
  print("Upload finished!")

  # Show the Data in the local Path
  print("\nListing files in local test directory")
  !ls -l $localFilePath
  
  
def createFileList():
  files = []
  # r=root, d=directories, f = files
  for r, d, f in os.walk(localFilePath):
      for file in f:
          if file.endswith(".jpg"):
              files.append(os.path.join(r, file))

  print("Printing files:")

  for f in files:
      print(f)
      
  return files

def prepareImagesForClassification(files):
  
  images = []
  
  for file in files:
    try:
      image = Image.open(file)
      
       #for InceptionV3 images need a size of 299 x299
      image = image.resize((224, 224), Image.LANCZOS)
      
      image = image.convert("RGB")
      
      image = np.asarray(image)
      
      images.append(image)
    except OSError:
      pass
  
  images = np.asarray(images)
  
  return images

def classifyImages(images, model):
  
  # preprocess the images to fit to the model
  images_preprocessed = preprocess_input(images)
  
  # use the model to classifi the images
  images_pred = model.predict(images_preprocessed, verbose=1)
  
  pred_results = decode_predictions(images_pred)
  
  return pred_results

def printResults(images, predictedClasses):
  
  for i in range(len(images)):
    plt.figure()
    plt.imshow(images[i])
    plt.title(predictedClasses[i])
    plt.axis('off')
    
  plt.show()
  
def compareResults(fileNames, resulsList, modelList, threshold):
  
  for i in range(len(fileNames)):
    print("Comparing the Results for File: " + fileNames[i])
    
    for j in range(5):
      
      for k in range(len(modelList)):
        
        resultToPrint = resultsList[k][i][j]
        
        # Setze 5% als Threshold
        if(resultToPrint is not None and resultToPrint[2] > threshold):
          print(" > ", (j+1), ". Platz @ ", modelList[k], " : ", resultsList[k][i][j])
        else:
          print(" > ", (j+1), ". Platz @ ", modelList[k], " : ", "--- Threshold-Filter ---")
      
      print("=================================================================")
      
    print("")

## Dateien übertragen -> einmal bei Session Start ausführen (bzw. bei neuen Bildern im Drive)

In [15]:
# Lade die Bilder erneut + File Preparation -> einmal beim Session Start ausführen
reloadPictures()
foundFiles = createFileList()
preparedImages = prepareImagesForClassification(foundFiles)




Deleted local directory

Calling the ListFolder method, that uploads the data...
* 11 files were uploaded in /root/data
Upload finished!

Listing files in local test directory
total 1636
-rw-r--r-- 1 root root 106647 Apr  8 08:53 auto.jpg
-rw-r--r-- 1 root root 204846 Apr  8 08:53 berge.jpg
-rw-r--r-- 1 root root 305634 Apr  8 08:53 bora-bora.jpg
-rw-r--r-- 1 root root 104237 Apr  8 08:53 brücke.jpg
-rw-r--r-- 1 root root 118569 Apr  8 08:53 handy.jpg
-rw-r--r-- 1 root root  51903 Apr  8 08:53 hund.jpg
-rw-r--r-- 1 root root  59945 Apr  8 08:53 katze.jpg
-rw-r--r-- 1 root root  53904 Apr  8 08:53 papagei.jpg
-rw-r--r-- 1 root root 204156 Apr  8 08:53 rose.jpg
-rw-r--r-- 1 root root  97310 Apr  8 08:53 sandwich.jpg
-rw-r--r-- 1 root root 346140 Apr  8 08:53 waffe.jpg
Printing files:
/root/data/katze.jpg
/root/data/waffe.jpg
/root/data/sandwich.jpg
/root/data/rose.jpg
/root/data/papagei.jpg
/root/data/auto.jpg
/root/data/bora-bora.jpg
/root/data/handy.jpg
/root/data/brücke.jpg
/root/data

## Bildklassifikation mit verschiedenen Modellen

In [22]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
#from keras.applications.inception_v3 import InceptionV3
#from keras.layers import Input
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.resnet50 import ResNet50

# VGG 16
predictedClassesVGG16 = classifyImages(preparedImages, VGG16(input_shape=(224, 224, 3)))
#printResults(preparedImages, predictedClassesVGG16)

# VGG 19
predictedClassesVGG19 = classifyImages(preparedImages, VGG19(input_shape=(224, 224, 3)))
#printResults(preparedImages, predictedClassesVGG19)


# InceptionV3 (works only if image input is 299x299)
#predictClassesInceptionV3 = classifyImages(preparedImages, InceptionV3(input_shape=(299, 299, 3)))
#model = InceptionV3(input_tensor=input_tensor, weights=None, include_top=True)
#printResults(preparedImages, predictedClassesVGG19)

# MobileNetV2
predictedClassesMobileNetV2 = classifyImages(preparedImages, MobileNetV2(input_shape=(224, 224, 3)))
#printResults(preparedImages, predictedClassesVGG19)

# MobileNetV2
predictedClassesResNet50 = classifyImages(preparedImages, ResNet50(input_shape=(224, 224, 3)))
#printResults(preparedImages, predictedClassesVGG19)


11/11 [==============================] - 6s 561ms/step


## Modell Vergleich

In [23]:
resultsList = [predictedClassesVGG16, predictedClassesVGG19, predictedClassesMobileNetV2, predictedClassesResNet50]

modelList = ['VGG16', 'VGG19', 'MobileNetV2', 'ResNet50']

# Diese Methode hier braucht 4 Parameter:
# 1. Die Liste der Dateinamen
# 2. Die Ergebnisse der einzelnen Modellen, als Array zusammengefasst
# 3. Die Namen der Modelle, einfach als String Array
# 4. Threshold (mind. Sicherheit der Modellvorhersage)
compareResults(foundFiles, resultsList, modelList, 0.05)

Comparing the Results for File: /root/data/katze.jpg
 >  1 . Platz @  VGG16  :  ('n02497673', 'Madagascar_cat', 0.3115199)
 >  1 . Platz @  VGG19  :  ('n02124075', 'Egyptian_cat', 0.5063072)
 >  1 . Platz @  MobileNetV2  :  ('n04209239', 'shower_curtain', 0.6135824)
 >  1 . Platz @  ResNet50  :  ('n02124075', 'Egyptian_cat', 0.3604978)
 >  2 . Platz @  VGG16  :  ('n02123159', 'tiger_cat', 0.25119665)
 >  2 . Platz @  VGG19  :  ('n02123159', 'tiger_cat', 0.20458318)
 >  2 . Platz @  MobileNetV2  :  --- Threshold-Filter ---
 >  2 . Platz @  ResNet50  :  ('n02123159', 'tiger_cat', 0.18962997)
 >  3 . Platz @  VGG16  :  ('n02124075', 'Egyptian_cat', 0.2346113)
 >  3 . Platz @  VGG19  :  ('n02127052', 'lynx', 0.17622991)
 >  3 . Platz @  MobileNetV2  :  --- Threshold-Filter ---
 >  3 . Platz @  ResNet50  :  ('n02127052', 'lynx', 0.17783482)
 >  4 . Platz @  VGG16  :  ('n02127052', 'lynx', 0.11448761)
 >  4 . Platz @  VGG19  :  ('n02497673', 'Madagascar_cat', 0.06959338)
 >  4 . Platz @  Mob